# Q8: Results

**Phase 9:** Results & Insights  
**Points: 3 points**

**Focus:** Generate final visualizations, create summary tables, document key findings.

**Lecture Reference:** Lecture 11, Notebook 4 ([`11/demo/04_modeling_results.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/04_modeling_results.ipynb)), Phase 9. Also see Lecture 07 (visualization).

---

## Setup

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load model results from Q7
predictions = pd.read_csv("output/q7_predictions.csv")
metrics = open("output/q7_model_metrics.txt").read()
feature_importance = pd.read_csv("output/q7_feature_importance.csv")

---

## Objective

Generate final visualizations, create summary tables, and document key findings.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q8_final_visualizations.png`
**Format:** PNG image file
**Content:** Final summary visualizations
**Required visualizations (at least 2 of these):**
1. **Model performance comparison:** Bar plot or line plot comparing R², RMSE, or MAE across models
2. **Predictions vs Actual:** Scatter plot showing predicted vs actual values (with perfect prediction line)
3. **Feature importance:** Bar plot showing top N features by importance
4. **Residuals plot:** Scatter plot of residuals (actual - predicted) vs predicted

**Requirements:**
- Clear axis labels (xlabel, ylabel)
- Title for each subplot
- Overall figure title (optional but recommended)
- Legend if multiple series shown
- Saved as PNG with sufficient resolution (dpi=150 or higher)

### 2. `output/q8_summary.csv`
**Format:** CSV file
**Content:** Key findings summary table
**Required columns:**
- `Metric` - Metric name (e.g., "R² Score", "RMSE", "MAE")
- One column per model (e.g., `Linear Regression`, `Random Forest`, `XGBoost`)

**Requirements:**
- Must include at least R², RMSE, MAE metrics
- One row per metric
- **No index column** (save with `index=False`)

**Example:**
```csv
Metric,Linear Regression,Random Forest,XGBoost
R² Score,-0.0201,0.9705,0.9967
RMSE,12.7154,2.1634,0.7276
MAE,9.8468,1.3545,0.4480
```

### 3. `output/q8_key_findings.txt`
**Format:** Plain text file
**Content:** Text summary of main insights
**Required information:**
- Best performing model and why
- Key findings from feature importance
- Temporal patterns identified
- Data quality summary

**Example format:**
```
KEY FINDINGS SUMMARY
===================

MODEL PERFORMANCE:
- Best performing model: XGBoost (R² = 0.9967)
- All models show reasonable performance (R² > 0.7 for tree-based models)
- XGBoost achieves lowest RMSE: 0.73°C

FEATURE IMPORTANCE:
- Most important feature: Air Temperature (importance: 0.6539)
- Top 3 features account for 93.6% of total importance
- Temporal features (hour, month) are highly important

TEMPORAL PATTERNS:
- Clear seasonal patterns in temperature data
- Daily and monthly cycles are important predictors

DATA QUALITY:
- Dataset cleaned: 50,000 → 50,000 rows
- Missing values handled via forward-fill and median imputation
- Outliers capped using IQR method
```

---

## Requirements Checklist

- [ ] Final visualizations created (model performance, key insights)
- [ ] Summary tables generated
- [ ] Key findings documented
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Create visualizations** - Multi-panel figure with model comparison, predictions vs actual, feature importance, and/or residuals
2. **Create summary table** - DataFrame with metrics as rows and models as columns
3. **Document key findings** - Text summary covering model performance, feature importance insights, temporal patterns, and data quality notes

---

## Decision Points

- **Visualizations:** What best communicates your findings? Model performance plots? Time series with predictions? Feature importance plots?
- **Summary:** What are the key takeaways? Document the most important findings from your analysis.

---

## Checkpoint

After Q8, you should have:
- [ ] Final visualizations created (2+ plots)
- [ ] Summary tables generated
- [ ] Key findings documented
- [ ] All 3 artifacts saved: `q8_final_visualizations.png`, `q8_summary.csv`, `q8_key_findings.txt`

---

**Next:** Continue to `q9_writeup.md` for Writeup.


## Generate Artifact 1 (q8_final_visualizations.png)

In [18]:
# Compute metrics directly from predictions
y_true = predictions["actual"]

model_cols = {
    "Linear Regression": "predicted_linear",
    "Random Forest": "predicted_random_forest",
    "XGBoost": "predicted_xgboost",
}

metrics = {}
for model_name, col in model_cols.items():
    y_hat = preds[col]
    mse = mean_squared_error(y_true, y_hat)
    metrics[model_name] = {
        "r2": r2_score(y_true, y_hat),
        "rmse": np.sqrt(mse),
        "mae": mean_absolute_error(y_true, y_hat),
    }

# Build the 3-panel figure
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Panel 1: Model Test R² Comparison
r2_values = [metrics[m]["r2"] for m in model_cols.keys()]
axes[0].bar(list(model_cols.keys()), r2_values)
axes[0].set_title("Model Test R² Comparison")
axes[0].set_ylabel("R²")
axes[0].set_ylim(0, 1)

# Panel 2: Predicted vs Actual (XGBoost)
axes[1].scatter(predictions["actual"], predictions["predicted_xgboost"], alpha=0.4)
min_val = min(predictions["actual"].min(), predictions["predicted_xgboost"].min())
max_val = max(predictions["actual"].max(), predictions["predicted_xgboost"].max())
axes[1].plot([min_val, max_val], [min_val, max_val], color="red")
axes[1].set_title("Predicted vs Actual (XGBoost)")
axes[1].set_xlabel("Actual")
axes[1].set_ylabel("Predicted")

# Panel 3: Top 10 Feature Importances
top10 = feature_importance.head(10)
axes[2].barh(top10["feature"], top10["importance"])
axes[2].invert_yaxis()
axes[2].set_title("Top 10 Feature Importances")
axes[2].set_xlabel("Importance")

plt.tight_layout()
plt.savefig("output/q8_final_visualizations.png", dpi=150)
plt.close()

print("Saved: output/q8_final_visualizations.png")

Saved: output/q8_final_visualizations.png


## Generate Artifact 2 (q8_summary.csv)

In [19]:
summary = {
    "Metric": ["R²", "RMSE", "MAE"],
    "Linear Regression": [
        metrics["Linear Regression"]["r2"],
        metrics["Linear Regression"]["rmse"],
        metrics["Linear Regression"]["mae"],
    ],
    "Random Forest": [
        metrics["Random Forest"]["r2"],
        metrics["Random Forest"]["rmse"],
        metrics["Random Forest"]["mae"],
    ],
    "XGBoost": [
        metrics["XGBoost"]["r2"],
        metrics["XGBoost"]["rmse"],
        metrics["XGBoost"]["mae"],
    ],
}

summary_df = pd.DataFrame(summary)
summary_df.to_csv("output/q8_summary.csv", index=False)

print("Saved: output/q8_summary.csv")

Saved: output/q8_summary.csv


## Generate Artifact 3 (q8_key_findings.txt)

In [ ]:
# Identify best model by highest test R²
best_model = max(metrics, key=lambda m: metrics[m]["r2"])
best_r2 = metrics[best_model]["r2"]
best_rmse = metrics[best_model]["rmse"]

# Top 5 feature importances
top_features = feature_importance.head(5)

# Convert to bullet list text
feature_lines = "\n".join(
    [
        f"- {row['feature']} (importance: {row['importance']:.4f})"
        for _, row in top_features.iterrows()
    ]
)

# Findings text
findings = f"""
KEY FINDINGS SUMMARY
====================

MODEL PERFORMANCE:
- Best performing model: {best_model} (Test R² = {best_r2:.4f})
- This model also achieves a low RMSE of {best_rmse:.2f}, indicating strong predictive accuracy.
- Linear Regression performed worst, while Random Forest performed well but slightly below XGBoost.

FEATURE IMPORTANCE:
Top 5 most influential predictors:
{feature_lines}

INSIGHTS:
- Tree-based models (Random Forest and XGBoost) outperform Linear Regression, suggesting important nonlinear relationships in the data.
- Feature importances indicate which environmental factors most strongly influence temperature predictions.
- Predictions vs actual values show that the best model captures patterns well with moderate residual spread.

DATA QUALITY SUMMARY:
- Missing values were handled using forward/backward fill and 0-filling for rain-related variables.
- Outliers were capped using the IQR method across all numeric features.
- Final cleaned dataset retained the full set of {len(predictions)} prediction rows.

"""

# Save findings text to q8_key_findings.txt
with open("output/q8_key_findings.txt", "w") as f:
    f.write(findings.strip())

print("Saved: output/q8_key_findings.txt")

Saved: output/q8_key_findings.txt
